In [ ]:
def is_sub_org(sub_org):
    for pfx in org_prefixes:
        if sub_org.startswith(pfx):
            return True
    return False
    
def add_sub_org(onto,sub_org):
    if sub_org.startswith('Departmentof'):
        sub_org = 'Stanford'+sub_org.strip('Departmentof')+'Department'
        if len(onto.search(iri='*'+sub_org)) == 0:
            sub_org_indiv = vivoNS.AcademicDepartment(sub_org)
            sub_org_indiv.subOrganizationWithin.append(school_indiv)
    elif sub_org.startswith('Divisionof'):
        sub_org = 'Stanford'+sub_org
        if len(onto.search(iri='*'+sub_org)) == 0:
            sub_org_indiv = vivoNS.Division(sub_org)
            sub_org_indiv.subOrganizationWithin.append(school_indiv)
    elif sub_org.startswith('Centerfor'):
        sub_org = 'Stanford' + sub_org
        if len(onto.search(iri='*'+sub_org)) == 0:
            sub_org_indiv = vivoNS.Center(sub_org)
            sub_org_indiv.subOrganizationWithin.append(school_indiv) 

def add_all_sub_orgs(onto,sub_org):
    print(sub_org)
    sub_sub_orgs = sub_org.split('and')
    first_com = sub_sub_orgs[0]
    if len(sub_sub_orgs) == 1:
        return add_sub_org(onto,first_com)
    for i in range(1,len(sub_sub_orgs)):
        if (is_sub_org(sub_sub_orgs[i])):
            add_sub_org(onto,first_com)
            add_all_sub_orgs(onto,sub_org[len(first_com)+3:])
            break
        else:
            first_com += 'and'
            first_com += sub_sub_orgs[i]
    add_sub_org(onto,first_com)
            
# add MESH terms and PubMed articles
org_prefixes = ['Departmentof','Divisionof','Centerof']
df = pd.read_csv('authors_cleaned.csv',sep=',',skipinitialspace=True,quoting=csv.QUOTE_ALL,engine='python')
for aff in df.afft.unique():
    if not 'Stanford University' in aff: continue
    locs = aff.split(',')
    if len(locs) < 1: continue
    if len(locs) == 1: 
        school = locs[0].strip()
        continue
    else:
        sub_org = locs[0].strip()
        school = locs[1].strip()
    if school == 'Stanford University':
        school_indiv = onto.search(iri='*StanfordUniversity')[0]
    elif school in ('School of Medicine at Stanford University','Stanford University School of Medicine'):
        school_indiv = onto.search(iri='*StanfordUniversitySchoolofMedicine')[0]
    elif school in ('Stanford University Medical Center'):
        school_indiv = onto.search(iri='*StanfordUniversityMedicalCenter')[0]
    else: continue #not affiliated with Stanford
        
    sub_org = sub_org.split('(')[0].split('-')[0].replace(' ','').strip('1]').replace('&','and').replace('/','and').split(';')[0]
    add_all_sub_orgs(onto,sub_org)
    
onto.save()

In [ ]:
org_prefixes = ['Departmentof','Divisionof','Centerof']

def parse_sub_org(aff):
    if not 'Stanford University' in aff: return None
    locs = aff.split(',')
    if len(locs) < 1: return None
    elif len(locs) == 1: return 'StanfordUniversity'
    else:
        sub_org = locs[0].strip()
        sub_org = sub_org.split('(')[0].split('-')[0].replace(' ','').strip('1]').replace('&','and').replace('/','and').split(';')[0]
        return sub_org

def is_sub_org(sub_org):
    for pfx in org_prefixes:
        if sub_org.startswith(pfx):
            return True
    return False
    
def get_sub_org(onto,sub_org,orgs_list):
#     print(orgs_list)
    if sub_org.startswith('Departmentof'):
        sub_org = 'Stanford'+sub_org.strip('Departmentof')+'Department'
    elif sub_org.startswith('Divisionof'):
        sub_org = 'Stanford'+sub_org
    elif sub_org.startswith('Centerfor'):
        sub_org = 'Stanford' + sub_org
    else: return
    indiv = onto.search_one(iri='*'+sub_org)
    print(indiv)
    orgs_list.append(indiv)
                            
def get_all_sub_orgs(onto,sub_org,orgs_list):
    sub_sub_orgs = sub_org.split('and')
    first_com = sub_sub_orgs[0]
    if len(sub_sub_orgs) == 1:
        return get_sub_org(onto,first_com,orgs_list)
    for i in range(1,len(sub_sub_orgs)):
        if (is_sub_org(sub_sub_orgs[i])):
            get_sub_org(onto,first_com,orgs_list)
            get_all_sub_orgs(onto,sub_org[len(first_com)+3:],orgs_list)
            break
        else:
            first_com += 'and'
            first_com += sub_sub_orgs[i]
    get_sub_org(onto,first_com,orgs_list)


df = pd.read_csv('authors_cleaned.csv',sep=',',skipinitialspace=True,quoting=csv.QUOTE_ALL,engine='python')
for index, line in df.iterrows():
    print(index)
    author = line['author']
    pmid = line['pmid']
    afft = line['afft']
    first_name = author.split(' ')[0]
    last_name = author.split(' ')[1]
    faculty_indiv = onto.search_one(iri='*'+first_name+last_name)
    if not faculty_indiv: faculty_indiv = onto.search_one(iri='*'+author.replace(' ',''))
    if not faculty_indiv: continue
    
#     print(author)
    sub_orgs = []
    sub_org_str = parse_sub_org(afft)
    if (sub_org_str == None): continue
    get_all_sub_orgs(onto,sub_org_str,sub_orgs)
    for sub_org in sub_orgs:
        try:
            if sub_org == None:
                sub_org = onto.search_one(iri='*StanfordUniversity')
    #         print(sub_org)
            if not sub_org in faculty_indiv.currentMemberOf:
                faculty_indiv.currentMemberOf.append(sub_org)
            article_name = 'PubMedArticle'+str(pmid)
            article_indiv = onto.search_one(iri='*'+article_name)
            if not article_indiv:
                article_indiv = vivoNS.AcademicArticle(article_name)
                article_indiv.pmid = pmid
            if article_indiv not in faculty_indiv.authorOf:
                faculty_indiv.authorOf.append(article_indiv)
        except:
            continue
onto.save()


In [ ]:
with open('old/results','r') as f:
    lines = f.readlines()
    inserts = 0
    for line in lines:
        if any(c.isalpha() for c in line):
            print(line)
            inserts += 1
print('Inserted data for '+str(inserts)+' person-pubMed relationships '+str(len(lines)-inserts)+' PubMed records')
print('Added 10 pubmed articles for students and 144 for faculty members')